# K-nearest neighbors: Phân tích toán học


#### Khoảng cách từ 1 điểm -> từng điểm trong một tập hợp

In [1]:
from __future__ import print_function
import numpy as np
from time import time       # for comparing runtimes
d, N = 1000, 1000           # dimension, number of training points
X = np.random.randn(N, d)   # N d-dimensional points
z = np.random.randn(d)      # the point we are starting from

In [3]:
# naively compute square distance between two vectors
def dist_pp(z, x):
    d = z - x.reshape(z.shape)   # force x and z to have the same dims
    return np.sum(d*d)

# from one point to each point in a set, naive
def dist_ps_naive(z, X):
    N = X.shape[0]
    res = np.zeros((1, N))
    for i in range(N):
        res[0][i] = dist_pp(z, X[i])
    return res

# from one point to each point in a set, fast
def dist_ps_fast(z, X):
    X2 = np.sum(X*X, 1)         # square of 12 norm of each ROW of X
    z2 = np.sum(z*z)            # square of 12 norm of z
    return X2 + z2 - 2*X.dot(z) # z2 can be ignored

t1 = time()
D1 = dist_ps_naive(z, X)
print('naive point2set, runtime: ', time() - t1, 's')

t1 = time()
D2 =  dist_ps_fast(z, X)
print('fast point2set , runtime: ', time() - t1, 's')
print('Result difference: ', np.linalg.norm(D1 - D2))

naive point2set, runtime:  0.02226877212524414 s
fast point2set , runtime:  0.012215137481689453 s
Result difference:  7.840271632433845e-12


#### Khoảng cách giữa từng cặp điểm-điểm trong hai tập hợp 
Thông thường, không những phải tính khoảng cách từ một điểm z tới tập các điểm X, mà phải tính giữa nhiều điểm Z tới X (= tính từng cặp khoảng cách giữa mỗi điểm trong tập kiểm thử và một điểm trong tập huấn luyện). 

Nếu không có phương pháp hiệu quả, runtime sẽ rất lớn.

In [4]:
M = 100
Z = np.random.randn(M, d)

# from each point in one set to each point in another set, half fast
def dist_ss_0(Z, X):
    M = Z.shape[0]
    N = X.shape[0]
    res = np.zeros((M, N))
    for i in range(M):
        res[i] = dist_ps_fast(Z[i], X)
    return res

# from each point in one set to each point in another set, fast
def dist_ss_fast(Z, X):
    X2 = np.sum(X*X, 1)         # square of 12 norm of each ROW of X
    Z2 = np.sum(Z*Z, 1)         # square of 12 norm of each ROW of Z
    return Z2.reshape(-1, 1) + X2.reshape(1, -1) - 2*Z.dot(X.T)

t1 = time()
D3 = dist_ss_0(Z, X)
print('half fast set2set runtime: ', time() - t1, 's')

t1 = time()
D4 = dist_ss_fast(Z, X)
print('fast set2set runtime: ', time() - t1, 's')
print('Result difference: ', np.linalg.norm(D3 - D4))

half fast set2set runtime:  0.3076169490814209 s
fast set2set runtime:  0.028321027755737305 s
Result difference:  3.232226962412253e-11
